In [1]:
import sys
sys.path.append('/home/meiri.yoav/biomed_proj/pyPPG')

In [2]:
from Prefiltering import*
from FiducialPoints import*
from Biomarkers2 import*
from Summary import*
from Statistics import*

import matplotlib
matplotlib.use('Agg')

import matplotlib.pyplot as plt
import numpy as np
import glob

In [3]:
annot_files = sorted(glob.glob("/home/meiri.yoav/biomed_proj/data/annotated/*"))
af_files = [f for f in annot_files if 'non' not in f]
non_af_files = [f for f in annot_files if 'non' in f]

In [4]:
#! REMEMBER TO EXCLUDE TPI, sample_idx, os
import pickle as pkl

def load_annotated(path):
    # example: path = '/data/home/meiri.yoav/biomed_proj/data/annotated/annot_af_001.pkl'
    with open(path, 'rb') as f:
        annot = pkl.load(f)
    hr = annot['hr']
    fiducials = annot['fiducials']
    str_num = annot['sample_num']
    fs = annot['fs']
    cond = annot['condition']
    osignal_data = annot['osignal_data']
    return cond, hr, fiducials, str_num, fs, osignal_data
    
# open /data/home/meiri.yoav/biomed_proj/data/annotated_AF/annot_AF_001.pkl
with open('/home/meiri.yoav/biomed_proj/data/annotated/annot_af_001.pkl', 'rb') as f:
    annot = pkl.load(f)

In [5]:
cond, hr, fiducials, str_num, fs, osignal_data = load_annotated('/home/meiri.yoav/biomed_proj/data/annotated/annot_af_001.pkl')

In [6]:
fiducials['pk'].dropna().values[:30]

<IntegerArray>
[  78,  165,  241,  345,  403,  480,  556,  645,  741,  825,  903, 1004, 1063,
 1150, 1253, 1308, 1398, 1468, 1541, 1613, 1701, 1765, 1846, 1938, 2025, 2108,
 2216, 2291, 2368, 2466]
Length: 30, dtype: Int64

In [7]:
size_in_cycles = 30

def plot_annotated_signal(fiducials):
    plt.figure(figsize=(40,5))
    plt.plot(hr[:fiducials['os'].dropna().values.astype(int)[size_in_cycles]],'k',linewidth=0.7)
    plt.plot(fiducials['pk'].dropna().values[:size_in_cycles], hr[fiducials['pk'].dropna().values.astype(int)[:size_in_cycles]], 'ro')
    plt.plot(fiducials['os'].dropna().values[:size_in_cycles], hr[fiducials['os'].dropna().values.astype(int)[:size_in_cycles]], 'bs')
    plt.plot(fiducials['dn'].dropna().values[:size_in_cycles], hr[fiducials['dn'].dropna().values.astype(int)[:size_in_cycles]], 'm*')

    plt.xlim(right=fiducials['os'].dropna().values.astype(int)[size_in_cycles], left=0)  # adjust the right leaving left unchanged
    plt.title(f'PPG signal with fiducials - {str_num}')
    plt.legend(['signal', 'peak','onset','dic.notch'])
    plt.xlabel('Sample (Fs='+str(fs)+' Hz)')
    plt.ylabel('Amplitude')
    plt.show()

for f in af_files[:3]:
    cond, hr, fiducials, str_num, fs, osignal_data = load_annotated(f)
    plot_annotated_signal(fiducials)

In [8]:
annot['osignal_data'] # features for each cycle, containing the sample index, os, Tpi of each fiducial point

,Tpi,Tpp,Tsys,Tdia,Tsp,Tdp,deltaT,Tsw10,Tsw25,Tsw33,...,Tpw33,Tpw50,Tpw66,Tpw75,Tpw90,Asp,Aoff,AUCpi,sample_idx,os
0,1.128,0.696,0.728,0.400,0.616,0.728,0.112,0.096,0.088,0.080,...,0.416,0.336,0.224,0.160,0.072,172.429008,-149.386388,-1.877154,141,142
1,0.608,0.608,0.304,0.304,0.184,0.304,0.120,0.144,0.128,0.112,...,0.400,0.352,0.296,0.256,0.080,226.133830,48.399666,10.349580,217,218
2,0.824,0.832,0.320,0.504,0.184,0.320,0.136,0.136,0.120,0.104,...,0.504,0.376,0.240,0.176,0.096,232.747573,-44.824472,2.326580,320,321
3,0.520,0.464,0.312,0.208,0.192,0.312,0.120,0.144,0.120,0.112,...,0.320,0.240,0.168,0.128,0.072,309.347675,96.751737,8.843955,385,386
4,0.560,0.616,0.264,0.296,0.136,0.264,0.128,0.104,0.088,0.080,...,0.280,0.192,0.128,0.104,0.064,112.818888,-67.290288,5.942068,455,456
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1731,0.736,0.640,0.376,0.360,0.224,0.376,0.152,0.176,0.144,0.128,...,0.248,0.184,0.144,0.112,0.072,495.697154,145.170741,9.999794,148841,148842
1732,0.528,0.664,0.248,0.280,0.128,0.248,0.120,0.104,0.088,0.088,...,0.344,0.288,0.256,0.232,0.096,109.657865,10.495864,10.010608,148907,148908
1733,0.824,0.696,0.448,0.376,0.264,0.448,0.184,0.216,0.144,0.128,...,0.304,0.224,0.176,0.160,0.104,139.540072,47.160745,9.997175,149010,149011
1734,0.496,0.736,0.352,0.144,0.136,0.352,0.216,0.096,0.080,0.072,...,0.240,0.208,0.168,0.136,0.072,81.423575,-59.161973,9.987521,149072,149073


Take a single 30 cycle window of the PPG signal according to the os column in the df and plot the signal

In [9]:
# plot hr from onsets 1 - 30 according to os column in annot['osignal_data']
left_idx = annot['osignal_data']['os'].dropna().values.astype(int)[0]
right_idx = annot['osignal_data']['os'].dropna().values.astype(int)[30]

cycle_signal_30 = hr[left_idx:right_idx]
print(cycle_signal_30[:50])

[630 659 682 702 717 726 735 742 747 749 748 744 738 729 719 708 695 682
 669 653 635 618 599 582 564 548 536 524 514 506 500 494 488 483 478 473
 468 463 458 454 450 445 439 435 430 425 420 415 410 404]


In [11]:
from hrvanalysis.extract_features import get_time_domain_features
from hrvanalysis.extract_features import get_frequency_domain_features
from hrvanalysis.extract_features import get_geometrical_features
from hrvanalysis.extract_features import get_poincare_plot_features
from hrvanalysis.extract_features import get_csi_cvi_features
import pandas as pd

''' reference: https://aura-healthcare.github.io/hrv-analysis/hrvanalysis.html'''

time_domain_features = get_time_domain_features(cycle_signal_30)
frequency_domain_features = get_frequency_domain_features(cycle_signal_30, sampling_frequency=fs)
geometrical_features = get_geometrical_features(cycle_signal_30)
poincare_plot_features = get_poincare_plot_features(cycle_signal_30)
csi_cvi_features = get_csi_cvi_features(cycle_signal_30)

# aggregate all features into one dict and turn it into a dataframe
features = {**time_domain_features, **frequency_domain_features, **geometrical_features, **poincare_plot_features, **csi_cvi_features}
features_arr = pd.DataFrame(features, index=[0]).dropna(axis=1).to_numpy()
# print(features_df.loc[:, features_df.isnull().any()].columns)
features_arr

array([[4.47226522e+02, 1.55176103e+02, 1.33744256e+01, 0.00000000e+00,
        0.00000000e+00, 2.69000000e+02, 1.08467742e+01, 1.33744460e+01,
        4.15000000e+02, 5.93000000e+02, 2.99053061e-02, 3.46974285e-01,
        1.51246647e+02, 2.84360190e+02, 7.46268657e+01, 5.17897555e+01,
        1.10448385e+00, 2.18678032e+01, 5.05073072e-02, 4.80789680e+00,
        9.51921032e+01, 2.29722870e+01, 0.00000000e+00, 3.75909091e+01,
        9.45905428e+00, 2.19248197e+02, 2.31786594e+01, 2.31786594e+01,
        4.52090373e+00, 2.03275171e+04]])

In [24]:
from hrv.rri import RRi
from hrv.classical import frequency_domain
from hrv.classical import time_domain
from hrv.classical import non_linear

def get_rri_from_cycle(cycle_signal):

    rri = RRi(cycle_signal, fs=fs)
    freeq_domain_dict = frequency_domain(rri)
    time_domain_dict = time_domain(rri)
    non_linear_dict = non_linear(rri)
    print(freeq_domain_dict)
    print(time_domain_dict)
    print(non_linear_dict)
    # aggregate all features to a single vector
    rri_features = np.array(list(freeq_domain_dict.values()) + list(time_domain_dict.values()) + list(non_linear_dict.values()))
    return rri_features

rri_features = get_rri_from_cycle(cycle_signal_30)
print(rri_features)

{'total_power': 20385.40292173149, 'vlf': 18513.766936364136, 'lf': 1847.6049813571606, 'hf': 24.031004010192213, 'lf_hf': 76.88421925998348, 'lfnu': 98.71604285245265, 'hfnu': 1.2839571475473397}
{'rmssd': 13.374446024101816, 'sdnn': 155.17610273762887, 'sdsd': 13.37712284877918, 'nn50': 0, 'pnn50': 0.0, 'mrri': 447.2265215638855, 'mhr': 151.24664696832357}
{'sd1': 9.459054279137266, 'sd2': 219.24819728750938}
[2.03854029e+04 1.85137669e+04 1.84760498e+03 2.40310040e+01
 7.68842193e+01 9.87160429e+01 1.28395715e+00 1.33744460e+01
 1.55176103e+02 1.33771228e+01 0.00000000e+00 0.00000000e+00
 4.47226522e+02 1.51246647e+02 9.45905428e+00 2.19248197e+02]


In [11]:
X_features = annot['osignal_data'].drop(columns=['Tpi', 'sample_idx', 'os']) # pure features for each cycle